In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 70.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 102.4 MB/s eta 0:00:00


In [ ]:
!pip3 install praw

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.4/189.4 kB 10.5 MB/s eta 0:00:00


In [ ]:
import os
import re
import string
import json
import numpy as np
import pandas as pd
from sklearn import metrics
import transformers
import praw
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, AutoTokenizer, BertModel, BertConfig, AutoModel, AdamW, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures
import tensorflow as tf

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df_train = pd.read_csv('train.tsv', delimiter='\t', header=None, names=['Text', 'Class', 'ID'])
df_dev = pd.read_csv('test.tsv', delimiter='\t', header=None, names=['Text', 'Class', 'ID'])

In [ ]:
df_train = pd.read_csv("/content/drive/MyDrive/Master/Human Computer Interaction/HCI Project/goemotions/data/train.tsv", sep='\t', header=None, names=['Text', 'Class', 'ID'])
df_dev = pd.read_csv("/content/drive/MyDrive/Master/Human Computer Interaction/HCI Project/goemotions/data/dev.tsv", sep='\t', header=None, names=['Text', 'Class', 'ID'])

In [ ]:
df_train['List of classes'] = df_train['Class'].apply(lambda x: x.split(','))
df_train['Len of classes'] = df_train['List of classes'].apply(lambda x: len(x))
df_dev['List of classes'] = df_dev['Class'].apply(lambda x: x.split(','))
df_dev['Len of classes'] = df_dev['List of classes'].apply(lambda x: len(x))

In [ ]:
with open('/content/drive/MyDrive/Master/Human Computer Interaction/HCI Project/goemotions/data/ekman_mapping.json') as file:
    ekman_mapping = json.load(file)

In [ ]:
with open('/content/ekman_mapping.json') as file:
    ekman_mapping = json.load(file)

In [ ]:
emotion_file = open("/content/drive/MyDrive/Master/Human Computer Interaction/HCI Project/goemotions/data/emotions.txt", "r")
emotion_list = emotion_file.read()
emotion_list = emotion_list.split("\n")
print(emotion_list)

['admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love', 'nervousness', 'optimism', 'pride', 'realization', 'relief', 'remorse', 'sadness', 'surprise', 'neutral']


In [ ]:
emotion_file = open("/content/emotions.txt", "r")
emotion_list = emotion_file.read()
emotion_list = emotion_list.split("\n")
print(emotion_list)

In [ ]:
def idx2class(idx_list):
    arr = []
    for i in idx_list:
        arr.append(emotion_list[int(i)])
    return arr

In [ ]:
df_train['Emotions'] = df_train['List of classes'].apply(idx2class)
df_dev['Emotions'] = df_dev['List of classes'].apply(idx2class)

In [ ]:
ekman_mapping

{'anger': ['anger', 'annoyance', 'disapproval'],
 'disgust': ['disgust'],
 'fear': ['fear', 'nervousness'],
 'joy': ['joy',
  'amusement',
  'approval',
  'excitement',
  'gratitude',
  'love',
  'optimism',
  'relief',
  'pride',
  'admiration',
  'desire',
  'caring'],
 'sadness': ['sadness', 'disappointment', 'embarrassment', 'grief', 'remorse'],
 'surprise': ['surprise', 'realization', 'confusion', 'curiosity']}

In [ ]:
def EmotionMapping(emotion_list):
    map_list = []
    
    for i in emotion_list:
        if i in ekman_mapping['anger']:
            map_list.append('negative')
        if i in ekman_mapping['disgust']:
            map_list.append('negative')
        if i in ekman_mapping['fear']:
            map_list.append('negative')
        if i in ekman_mapping['joy']:
            map_list.append('positive')
        if i in ekman_mapping['sadness']:
            map_list.append('negative')
        if i in ekman_mapping['surprise']:
            map_list.append('positive')
        if i == 'neutral':
            map_list.append('neutral')
            
    return map_list

In [ ]:
df_train['Mapped Emotions'] = df_train['Emotions'].apply(EmotionMapping)
df_dev['Mapped Emotions'] = df_dev['Emotions'].apply(EmotionMapping)

In [ ]:
df_train['positive'] = np.zeros((len(df_train),1))
df_train['negative'] = np.zeros((len(df_train),1))
df_train['neutral'] = np.zeros((len(df_train),1))

df_dev['positive'] = np.zeros((len(df_dev),1))
df_dev['negative'] = np.zeros((len(df_dev),1))
df_dev['neutral'] = np.zeros((len(df_dev),1))

In [ ]:
for i in ['positive', 'negative','neutral']:
    df_train[i] = df_train['Mapped Emotions'].apply(lambda x: 1 if i in x else 0)
    df_dev[i] = df_dev['Mapped Emotions'].apply(lambda x: 1 if i in x else 0)


df_train['class'] = df_train['Mapped Emotions'].apply(lambda x: 0 if ('positive' in x or 'neutral' in x) else 1)
df_dev['class'] = df_train['Mapped Emotions'].apply(lambda x: 0 if ('positive' in x or 'neutral' in x) else 1)

In [ ]:
df_train

,Text,Class,ID,List of classes,Len of classes,Emotions,Mapped Emotions,positive,negative,neutral,class
0,My favourite food is anything I didn't have to...,27,eebbqej,[27],1,[neutral],[neutral],0,0,1,0
1,"Now if he does off himself, everyone will thin...",27,ed00q6i,[27],1,[neutral],[neutral],0,0,1,0
2,WHY THE FUCK IS BAYLESS ISOING,2,eezlygj,[2],1,[anger],[negative],0,1,0,1
3,To make her feel threatened,14,ed7ypvh,[14],1,[fear],[negative],0,1,0,1
4,Dirty Southern Wankers,3,ed0bdzj,[3],1,[annoyance],[negative],0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...
43405,Added you mate well I’ve just got the bow and ...,18,edsb738,[18],1,[love],[positive],1,0,0,0
43406,Always thought that was funny but is it a refe...,6,ee7fdou,[6],1,[confusion],[positive],1,0,0,0
43407,What are you talking about? Anything bad that ...,3,efgbhks,[3],1,[annoyance],[negative],0,1,0,1
43408,"More like a baptism, with sexy results!",13,ed1naf8,[13],1,[excitement],[positive],1,0,0,0


In [ ]:
model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def convert_data_to_examples(train, test, DATA_COLUMN, LABEL_COLUMN): 
  train_InputExamples = train.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x[DATA_COLUMN], 
                                                          text_b = None,
                                                          label = x[LABEL_COLUMN]), axis = 1)

  validation_InputExamples = test.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x[DATA_COLUMN], 
                                                          text_b = None,
                                                          label = x[LABEL_COLUMN]), axis = 1)
  
  return train_InputExamples, validation_InputExamples

  train_InputExamples, validation_InputExamples = convert_data_to_examples(train, 
                                                                           test, 
                                                                           'DATA_COLUMN', 
                                                                           'LABEL_COLUMN')
  
def convert_examples_to_tf_dataset(examples, tokenizer, max_length=128):
    features = [] # -> will hold InputFeatures to be converted later

    for e in examples:
        # Documentation is really strong for this method, so please take a look at it
        input_dict = tokenizer.encode_plus(
            e.text_a,
            add_special_tokens=True,
            max_length=max_length, # truncates if len(s) > max_length
            return_token_type_ids=True,
            return_attention_mask=True,
            pad_to_max_length=True, # pads to the right by default # CHECK THIS for pad_to_max_length
            truncation=True
        )

        input_ids, token_type_ids, attention_mask = (input_dict["input_ids"],
            input_dict["token_type_ids"], input_dict['attention_mask'])

        features.append(
            InputFeatures(
                input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, label=e.label
            )
        )

    def gen():
        for f in features:
            yield (
                {
                    "input_ids": f.input_ids,
                    "attention_mask": f.attention_mask,
                    "token_type_ids": f.token_type_ids,
                },
                f.label,
            )

    return tf.data.Dataset.from_generator(
        gen,
        ({"input_ids": tf.int32, "attention_mask": tf.int32, "token_type_ids": tf.int32}, tf.int64),
        (
            {
                "input_ids": tf.TensorShape([None]),
                "attention_mask": tf.TensorShape([None]),
                "token_type_ids": tf.TensorShape([None]),
            },
            tf.TensorShape([]),
        ),
    )

DATA_COLUMN = 'Text'
LABEL_COLUMN = 'class'

In [ ]:
train_InputExamples, validation_InputExamples = convert_data_to_examples(df_train, df_dev, DATA_COLUMN, LABEL_COLUMN)

train_data = convert_examples_to_tf_dataset(list(train_InputExamples), tokenizer)
train_data = train_data.shuffle(100).batch(32).repeat(2)

validation_data = convert_examples_to_tf_dataset(list(validation_InputExamples), tokenizer)
validation_data = validation_data.batch(32)

/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')])

model.fit(train_data, epochs=2, validation_data=validation_data)

Epoch 1/2
2714/2714 [==============================] - 2278s 816ms/step - loss: 0.2651 - accuracy: 0.8825 - val_loss: 1.1758 - val_accuracy: 0.6576
Epoch 2/2
2714/2714 [==============================] - 2207s 813ms/step - loss: 0.0998 - accuracy: 0.9624 - val_loss: 1.7865 - val_accuracy: 0.6891


In [ ]:
model.save_pretrained('/content')

In [ ]:
# Initialize the Reddit instance with your credentials
reddit = praw.Reddit(client_id='wR1bOBPejs4v5grZsCk2ww',
                     client_secret='WSgVBz_Wlv0Cno4iXXwmv08eX3Zdwg',
                     username='adpat0324',
                     password='Krishrocks1',
                     user_agent='Mac:my_reddit_app:v1.0 by /u/adpat0324')

In [ ]:
post = reddit.submission(url="https://www.reddit.com/r/bipolar/comments/12o9lr9/toxic_environments/")
post_text = post.selftext

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



In [ ]:
tf_batch = tokenizer(post_text, max_length=128, padding=True, truncation=True, return_tensors='tf')
tf_outputs = model(tf_batch)
tf_predictions = tf.nn.softmax(tf_outputs[0], axis=-1)
labels = ['Positive','Negative']
label = tf.argmax(tf_predictions, axis=1)
label = list(label.numpy())
print(post_text, ": \n", labels[label[0]])
#for i in range(len(texts)):
  #print(texts[i], ": \n", labels[label[i]])

So I’m moving back to a place I spent most of my life as a child. I’ve been visiting more and o now realize that every time I leave there I feel upset and down on myself. The move is already in motion and too permanent to back out. There’s family around that do not respect boundaries and are extremely disrespectful gross human beings. I am so worried about being the asshole when it comes to setting boundaries. But on top of that am I going to feel sad everyday living there? How do you cope with toxic situations? 

A lot of the times the toxic people make me feel like it’s my fault/like I’m the bad person and it was only until a few years ago that I realized I am not. I process it in therapy and I have so many wonderful friends (my chosen family) that also point out when I’m being treated poorly. I grew up in such toxicity that I don’t know what I deserve and I’m afraid to stand up for myself for fear of being ‘the bitch’. : 
 Negative


In [ ]:
post2 = reddit.submission(url="https://www.reddit.com/r/happy/comments/12q3g19/my_crush_i_went_to_an_amusement_park_together_d/")
post_text2 = post2.selftext

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



In [ ]:
tf_batch = tokenizer(post_text2, max_length=128, padding=True, truncation=True, return_tensors='tf')
tf_outputs = model(tf_batch)
tf_predictions = tf.nn.softmax(tf_outputs[0], axis=-1)
labels = ['Positive','Negative']
label = tf.argmax(tf_predictions, axis=1)
label = list(label.numpy())
print(post_text2, ": \n", labels[label[0]])

The other day I went to an amusement park with the girl I like (for privacy sake I’ll call her Jane). It was originally supposed to be for a class trip, and I had invited her along, however my class cancelled due to the prices. We decided to go nevertheless, just the two of us. It was a three hour drive from where we live, so we had to leave at 6am; which meant I only ended up getting around 5-6 hours of sleep the night before. I was wide awake though, and the drive there was easy. We talked for a bit, but for the most part I just listened to music while she tried to get some sleep. When we got there around 9:30 we were greeted by a park employee who took our photo and popped the question, “I have to ask, are you two dating?” I shook my head. I was too caught off guard to notice Jane’s reaction. Was not expecting such a question to just come out of nowhere. It didn’t seem to have any effect though. It didn’t make things awkward, or at least I hope it didn’t on her behalf. 

Fast forwar

In [ ]:
texts = ["Thats so unfortunate", 
         "A surprise turn of events! I'm so glad you heard such great things about yourself!"]